#  Poppy-Right-Arm - Simulator version


The Poppy Right Arm is a project that is focused on modeling the effects of toy interaction on infant development.
For example, if given a rattle to shake, how does an infant develop motor skill such as rhythm? Are there any effects on an infants reactionss to sounds in general, and perhaps is there any effect on early language development? All of these questions are relevent to this project.

This version of the notebook is for working with the V-rep simulator.
A scene for the poppy right arm has not been developed yet, so using the stationary poppy humanoid is needed in its place.
As the physical robot can only have sensor data collected from the joints via sensors on the motor, this is useful for modeling the specifics of the hand rather than the rist in spatial relativity tothe head.


## Imports and general setup

In [ ]:
# needed for plotting within a notebook cell
%pylab inline

import time    # library for time based functionality

import numpy as np    # Python's linear algrabra library - needed for static arrays 

# Matplotlib is one of Python's standard library for plotting data
from mpl_toolkits.mplot3d import Axes3D    # This first library is specifically for plotting on a 3D plane
import matplotlib.pyplot as plt    # This is the general matplotlib library

# These are needed for the actual robot
import pypot.robot    # This is needed for the physical robot
from poppy.creatures import PoppyHumanoid    # The humanoid library is needed in order to use the simulator

poppy = PoppyHumanoid(simulator='vrep')    # initialization for the robot in the simulator

# Preps the motors for use
for m in poppy.motors:
    m.compliant = False    # makes all the joints in the robot stiff, which is better for holding positions
                           # making them flexible is for more human-robot interactions
    m.goto_behavior = 'minjerk'    # minimizes jerking motions - makes movements as smooth as possible
                                   # Folows Hogan and Flash's minimum jerk hypothesis
                                   # For more information,see: http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/MinimumJerkHypothesis.ipynb

            
# Python (array) lists to store feedback data
pos_Head = []    # Stores the position of the hand (center of palm) relative to the head
pos_Stand = []    # Stores the position of the hand (center of palm) relative to the center point between the feet
TIME = []    # Stores timestamps of when the arm is in a certain position
sys_load = []    # Stores the torque the motors are going through at a point in time.
                 # Good for being warry of the system load of a task and possibly calculating fatigue
speed = []    # Stores the angle speed the motors are travelling at
temp = []   # Stores the motors' temperature
volt = []   # Stores the motor's voltage. 
            # In conjunction with Temperaure, this can be used to measure energy of a task

# Motors

poppy.r_shoulder_x:
    * id: 52
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: -90.0 to 0.0 degrees 

poppy.r_shoulder_y
    * id: 51
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: -50.0 to 170.0 degrees 
    
poppy.r_arm_z
    * id: 53
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: -20.0 to 95.0 degrees 

poppy.r_elbow_y
    * id: 54
    * motor type: Dynamixel MX-28
    * orientation: indirect
    * angle offset: 0.0 degrees
    * angle limit: 0.0 to 130.0 degrees 

## A number of function commands for the robot

In [ ]:
def rest_position():
    """
    This command function returns the arm to a "resting" position, although it is more on the lines of the 
    attention position.
    
    For the physical arm, it is important to calibrate the motors so that the resting position is set for all the
    motors is at angle 0. The simulator defaults to this, so this will eliminate the need to constantly 
    translate angles when switching between the two.
    """
    poppy.r_shoulder_x.goal_position = 0    # Controls the motor
    poppy.r_shoulder_y.goal_position = 0
    poppy.r_arm_z.goal_position = 0
    poppy.r_elbow_y.goal_position = 0
    

In [ ]:
def app():
    pos_Head.append(poppy.get_object_position('r_forearm_visual','head_visual'))
    pos_Stand.append(poppy.get_object_position('r_forearm_visual'))
    
    sys_load.append(poppy.r_shoulder_x.present_load)
    sys_load.append(poppy.r_shoulder_y.present_load)
    sys_load.append(poppy.r_arm_z.present_load)
    sys_load.append(poppy.r_elbow_y.present_load)
    
    speed.append(poppy.r_shoulder_x.present_speed)
    speed.append(poppy.r_shoulder_y.present_speed)
    speed.append(poppy.r_arm_z.present_speed)
    speed.append(poppy.r_elbow_y.present_speed)
    
    temp.append(poppy.r_shoulder_x.present_temperature)
    temp.append(poppy.r_shoulder_y.present_temperature)
    temp.append(poppy.r_arm_z.present_temperature)
    temp.append(poppy.r_elbow_y.present_temperature)
     
    volt.append(poppy.r_shoulder_x.present_voltage)
    volt.append(poppy.r_shoulder_y.present_voltage)
    volt.append(poppy.r_arm_z.present_voltage)
    volt.append(poppy.r_elbow_y.present_voltage)

In [ ]:
def hand_wave(sec=10, rest=0.5):
    """"""
    t0 = time.time()
    while True:
        
        t1 = time.time()
        if t1-t0 >= sec:
            break

        # run for sec
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = 30
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -135
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = -120;
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = 0
        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
    
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = 45
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -175
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = -120
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = -65
        
        app()
        TIME.append(t1-t0)
        
        time.sleep(rest)
        app()
        TIME.append(t1-t0)

In [ ]:
def hand_shake(sec=3, rest=0.5):

    t0 = time.time()
    
    while True:
        
        t1 = time.time()
        if t1-t0 >= sec:
            break 
        
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = 35
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -135
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = 90
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = 25
        time.sleep(rest-.01)
        
        app()
        poppy.r_shoulder_y.goal_position = 50
        TIME.append(t1-t0)
        time.sleep(rest+0.1)
        
        

In [ ]:
rest_position()
# reset arrays
pos_Head = []
pos_Stand = []
TIME = []
sys_load = []
speed = []
temp = []
volt = []

In [ ]:
hand_wave()
time.sleep(0.25)
rest_position()

In [ ]:
hand_shake()
time.sleep(0.25)
rest_position()

In [ ]:
print len(pos_Head)
print len(pos_Stand)
print len(TIME)


In [ ]:
# formatting into arrays
pos_Head_Arr = np.asarray(pos_Head)
pos_Stand_Arr = np.asarray(pos_Stand)
sys_load_Arr = np.asarray(sys_load)
speed_Arr = np.asarray(speed)
temp_Arr = np.asarray(temp)
volt_Arr = np.asarray(volt)
TIME_Arr = np.asarray(TIME)


sys_load_Arr = sys_load_Arr.reshape((sys_load_Arr.size / 4,4))
speed_Arr = speed_Arr.reshape((speed_Arr.size / 4,4))
temp_Arr = temp_Arr.reshape((temp_Arr.size / 4,4))
volt_Arr = volt_Arr.reshape((volt_Arr.size / 4, 4))

In [ ]:
print pos_Head_Arr.size
print TIME_Arr.size
print speed_Arr[:,0].size

In [ ]:
standard = axes(projection='3d')
standard.scatter(*array(pos_Head_Arr).T)

In [ ]:
pos_Head_Arr

In [ ]:
head = axes(projection='3d')
head.scatter(*array(pos_Stand_Arr).T)

## Run when experiments are done

This cell resets the robot and releases the robot.
After running this set of commands, rerunning the setup cell is necessary

### **NOTE**
If this fails or some error occurs, it may be necessary to run a hard reset:
    
    For physical robot:
        1. Unplug the robot
    
    For simulator:
        1.Close the simulator
    
    2. Save and shutdown the ipython notebook
    3. It may be a good idea to also close the jupyter-notebook server and restart from there.

In [ ]:
reached_pt_Stand = []
reached_pt_Head = []
for m in poppy.motors:
    m.compliant = True
time.sleep(2)
poppy.stop_sync()
poppy.close()